In [16]:
import asyncio
from datetime import datetime
import os
from typing import Dict
from typing import Iterable
from typing import List
from typing import Optional

from dotenv import find_dotenv
from dotenv import load_dotenv
import instructor
from openai import AsyncAzureOpenAI
from pydantic import BaseModel
from pydantic import Field

In [17]:
load_dotenv(find_dotenv())
DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
)
client = instructor.patch(client)

In [90]:
class Task(BaseModel):
    id: int = Field(description="Monotonically increasing integer")
    task_name: str = Field("The camelcase classname of the task")
    depends_on: List[int] = Field(
        description="The task ids that need to finished prior to this task execution"
    )


class ListEmails(Task):
    """List all emails from an email correspondence between `participants`
    from `from_time` to `to_time`"""

    participants: List[str] = Field(description="The participants of the email correspondence")
    from_time: Optional[datetime]
    to_time: Optional[datetime]


class Summarize(Task):
    """Summarize or analyze the input data"""


class WriteEmail(Task):
    """A task that needs to be executed using email"""

    recipient: str


class ListTickets(Task):
    """List all the tickets from `project` filtered
    by `names`, `last_updated_after` or `deadline_up_to`"""

    project: str = Field(description="An uppercase project name")
    names: Optional[List[str]] = Field(description="Ticket names to list")
    last_updated_after: Optional[datetime]
    deadline_up_to: Optional[datetime]


class CreateTicket(Task):
    """Create a Jira ticket inside `project` assigned to `assignee`
    that needs to be completed before `deadline`"""

    name: str
    project: str = Field(description="An uppercase project name")
    assignee: Optional[str]
    deadline: Optional[datetime]


class Prompt(BaseModel):
    prompt: str
    chain_of_thoughs: str = Field(
        description="Step by step reasoning about the tasks to be performed"
    )
    tasks: List[ListEmails | ListTickets | WriteEmail | CreateTicket | Summarize] = Field(
        """The list of tasks created from the query."""
    )


async def list_tasks(
    query: List[Dict[str, str]],
    response_model: BaseModel | Iterable[BaseModel],
    stream: bool = False,
    client: AsyncAzureOpenAI = client,
) -> BaseModel | Iterable[BaseModel]:
    return await client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": """\
            You are an AI powered agent. You have access to following tools:
            - ListEmails
            - WriteEmail
            - Summarize
            - ListTickets
            - CreateTicket
            You are going to be prompted by the user.
            You are to decompose the prompt into a list of tasks that solves the initial prompt.
            Today is 2024-03-13, 15:07 o'clock. The name of the user is Victor.
            """,
            },
            {
                "role": "user",
                "content": query,
            },
        ],
        response_model=response_model,
        temperature=0,
        stream=stream,
    )

In [91]:
queries = [
    "Summarize the updates on tickets from ProjectA that happened over night.",
    "How many story points are still pending till the end of the week for the projectb?",
    (
        "Are there any inconsistencies left between the solution described under PROJECTA-345 and"
        "the plan I discussed with Matt via email last week?"
    ),
    (
        "Create a ticket to investigate instructor library inside the TICKET project, "
        "assign it to Nick and notify him. Set the deadline to the end of the week."
    ),
]

In [92]:
results = await asyncio.gather(*[list_tasks(q, Prompt) for q in queries])

In [93]:
for result in results:
    print(result.model_dump_json(indent=4))

{
    "prompt": "Summarize the updates on tickets from ProjectA that happened over night.",
    "chain_of_thoughs": "To summarize the updates on tickets from ProjectA that happened overnight, I need to first list all the tickets from ProjectA that were updated after the end of the previous day. Since today is 2024-03-13, the end of the previous day is 2024-03-12 23:59:59. After listing the tickets, I will then summarize the updates on these tickets.",
    "tasks": [
        {
            "id": 1,
            "task_name": "ListTickets",
            "depends_on": [],
            "project": "PROJECTA",
            "names": null,
            "last_updated_after": "2024-03-12T23:59:59Z",
            "deadline_up_to": null
        },
        {
            "id": 2,
            "task_name": "Summarize",
            "depends_on": [
                1
            ]
        }
    ]
}
{
    "prompt": "How many story points are still pending till the end of the week for the projectb?",
    "chain_of